In [3]:
from thalamus_merfish_analysis import abc_load_base as abc

In [11]:
cells_df = abc.get_combined_metadata(version='20231215')
len(cells_df)

3739961

## Filter by neurotransmitter and explore CCF region counts

In [ ]:
obs_dopa = cells_df.loc[cells_df['neurotransmitter']=='Dopa']
len(obs_dopa)

8352

In [17]:
dopa_region_counts = obs_dopa['parcellation_substructure'].value_counts().loc[lambda x: x>50]
dopa_region_counts

parcellation_substructure
MB-unassigned      1467
MOB-unassigned     1046
onl                 881
VTA                 644
OLF-unassigned      362
SNr                 270
MOBopl              258
IF                  254
HY-unassigned       239
SNc                 230
PAG-unassigned      211
CLI                 134
MRN                 130
ZI                  128
DR                  105
PVpo                 98
DMH                  93
ARH                  92
ml                   92
PH                   90
SUM                  88
rust-unassigned      69
RR                   64
RL                   59
PVi                  54
IPDM                 53
unassigned           52
Name: count, dtype: int64

## filter by CCF structure and load gene expression

In [26]:
dopa_substructures = dopa_region_counts.loc[lambda x: x>75].index
cells_dopa_ccf_subset = cells_df[cells_df['parcellation_substructure'].isin(dopa_substructures)]
len(cells_dopa_ccf_subset)

281868

In [27]:
adata = abc.load_adata(transform='raw', from_metadata=cells_dopa_ccf_subset)

In [28]:
len(adata)

281868

## Other filtering options

In [ ]:
cells_df = abc.filter_by_ccf_region(obs, regions)

In [ ]:
cells_df = abc.filter_by_class(obs)